In [1]:
import os
import json
from dotenv import load_dotenv
import csv
from io import BytesIO

from google.cloud import storage

In [2]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = str(os.environ["GOOGLE_APPLICATION_CREDENTIALS_PATH"])

GCS_BUCKET = os.environ.get("GCS_BUCKET_NAME")
bucket = storage.Client().bucket(GCS_BUCKET)

base_url = "https://storage.googleapis.com"

In [3]:
preset_folder = "./dataset/room_dataset"

In [4]:
def get_descriptions(csv_path, image_name):
    with open(csv_path, mode='r', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        for row in reader:
            if row['Image Name'].strip() == image_name:
                wall_desc = row['Wall Description'].strip()
                floor_desc = row['Floor Description'].strip()
                return wall_desc, floor_desc
    return None, None  # If image name not found

In [5]:
def upload_to_gcs(source_file_name, destination_blob_name):
    blob = bucket.blob(destination_blob_name)
    blob.upload_from_filename(source_file_name)
    print(f"File {source_file_name} uploaded to {destination_blob_name}.")

In [6]:
def upload_json_to_gcs(destination_blob_name, data_dict):
    blob = bucket.blob(destination_blob_name)
    json_bytes = BytesIO(json.dumps(data_dict).encode('utf-8'))
    blob.upload_from_file(json_bytes, content_type='application/json')
    print(f"Uploaded to gs://{destination_blob_name}")

In [7]:
for st in os.listdir(preset_folder):
    full_path = os.path.join(preset_folder, st)
    if os.path.isdir(full_path):
        pass
    else:
        name_without_ext = os.path.splitext(st)[0]
        image_folder = f"{preset_folder}/{name_without_ext}"
        image_name_list = os.listdir(image_folder)
        # print(image_name_list)
        print(name_without_ext)
        for image_name in image_name_list:
            image_name_without_ext = os.path.splitext(image_name)[0]
            wall_desc, floor_desc = get_descriptions(full_path, image_name_without_ext)
            # print(f"Wall Description: {wall_desc}")
            # print(f"Floor Description: {floor_desc}")

            image_path = f"{image_folder}/{image_name}"
            print(image_path)
            name_without_ext_cleaned = name_without_ext.replace(" ", "")
            image_destination_blob_name = 'ai-assistant/preset/'+str(name_without_ext_cleaned)+'/'+str(image_name)
            upload_to_gcs(source_file_name=image_path, destination_blob_name=image_destination_blob_name)
            image_url = f"{base_url}/{GCS_BUCKET}/{image_destination_blob_name}"
            print(image_url)

            desc_data = {
                'image_id': image_name_without_ext,
                'image_path': image_url,
                'wall_desc': wall_desc,
                'floor_desc': floor_desc,
                'style': str(name_without_ext)
            }
            desc_destination_blob_name = 'ai-assistant/preset/'+str(name_without_ext_cleaned)+'/desc_'+str(image_name_without_ext)+'.json'
            print(desc_destination_blob_name)
            upload_json_to_gcs(desc_destination_blob_name, desc_data)


Art Deco
./dataset/room_dataset/Art Deco/artdeco_1.jpg
File ./dataset/room_dataset/Art Deco/artdeco_1.jpg uploaded to ai-assistant/preset/ArtDeco/artdeco_1.jpg.
https://storage.googleapis.com/weshape-dev/ai-assistant/preset/ArtDeco/artdeco_1.jpg
ai-assistant/preset/ArtDeco/desc_artdeco_1.json
Uploaded to gs://ai-assistant/preset/ArtDeco/desc_artdeco_1.json
./dataset/room_dataset/Art Deco/artdeco_2.jpg
File ./dataset/room_dataset/Art Deco/artdeco_2.jpg uploaded to ai-assistant/preset/ArtDeco/artdeco_2.jpg.
https://storage.googleapis.com/weshape-dev/ai-assistant/preset/ArtDeco/artdeco_2.jpg
ai-assistant/preset/ArtDeco/desc_artdeco_2.json
Uploaded to gs://ai-assistant/preset/ArtDeco/desc_artdeco_2.json
./dataset/room_dataset/Art Deco/artdeco_3.jpg
File ./dataset/room_dataset/Art Deco/artdeco_3.jpg uploaded to ai-assistant/preset/ArtDeco/artdeco_3.jpg.
https://storage.googleapis.com/weshape-dev/ai-assistant/preset/ArtDeco/artdeco_3.jpg
ai-assistant/preset/ArtDeco/desc_artdeco_3.json
Upload

In [ ]:
# from concurrent.futures import ThreadPoolExecutor

# bucket_name = GCS_BUCKET
# sub_folder = "ai-assistant/preset"

# client = storage.Client()
# prefix = sub_folder if sub_folder.endswith("/") else sub_folder + "/"
# blobs = client.list_blobs(bucket_name, prefix=prefix)

# # Filter only JSON blobs first
# json_blobs = [blob for blob in blobs if blob.name.endswith('.json')]

# def download_and_parse(blob):
#     try:
#         json_data = blob.download_as_text()
#         data = json.loads(json_data)
#         return {
#             'image_id': data['image_id'],
#             'image_path': data['image_path']
#         }
#     except Exception as e:
#         print(f"Failed to process {blob.name}: {e}")
#         return None

# preset_list = []
# with ThreadPoolExecutor(max_workers=10) as executor:
#     results = executor.map(download_and_parse, json_blobs)
#     preset_list = [r for r in results if r is not None]

# print(preset_list)